In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, Add, TimeDistributed
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

# Load and preprocess the new dataset
path = "/content/AEP_hourly.csv"
data = pd.read_csv(path, parse_dates=['Datetime'])

# Set Datetime as index
data.set_index('Datetime', inplace=True)

# Handle missing values
data.fillna(method='ffill', inplace=True)

# Normalize features
scalers = {}
features = ['Temperature', 'Relative Humidity', 'Wind Speed', 'Precipitation', 'Is_Weekend_Holiday', 'AEP_MW']
for feature in features:
    scaler = MinMaxScaler()
    data[feature] = scaler.fit_transform(data[[feature]])
    scalers[feature] = scaler

# Prepare sequences
n_timesteps = 24
X, Y = [], []
for i in range(len(data) - n_timesteps):
    X.append(data.iloc[i:i + n_timesteps][['Temperature', 'Relative Humidity', 'Wind Speed', 'Precipitation', 'Is_Weekend_Holiday']].values)
    Y.append(data.iloc[i + n_timesteps]['AEP_MW'])
X, Y = np.array(X), np.array(Y)

# Split into training and testing
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
Y_train, Y_test = Y[:split], Y[split:]

<ipython-input-1-f14d58edd551>:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [2]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-Head Self Attention
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = Dropout(dropout)(x)
    x = Add()([x, inputs])
    x = LayerNormalization(epsilon=1e-6)(x)

    # Feed Forward Part
    x_ff = Dense(ff_dim, activation="relu")(x)
    x_ff = Dropout(dropout)(x_ff)
    x_ff = Dense(inputs.shape[-1])(x_ff)
    x = Add()([x, x_ff])
    x = LayerNormalization(epsilon=1e-6)(x)
    return x

def build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = Input(shape=input_shape)
    x = inputs

    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = TimeDistributed(Dense(mlp_units, activation="relu"))(x)
    x = Dropout(mlp_dropout)(x)
    outputs = Dense(1)(x)

    return Model(inputs, outputs)

input_shape = (n_timesteps, X_train.shape[2])
model = build_transformer_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=128,
    dropout=0.1,
    mlp_dropout=0.1,
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="mse", metrics=["mae"])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 24, 5)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 24, 5)          │         23,557 │ input_layer[0][0],     │
│ (MultiHeadAttention)      │                        │                │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 24, 5)          │              0 │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 24, 5)          │              0 │ dropout_1[0][0],       │
│                           │                        │                │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 24, 5)          │             10 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 24, 4)          │             24 │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 24, 4)          │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 24, 5)          │             25 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 24, 5)          │              0 │ layer_normalization[0… │
│                           │                        │                │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 24, 5)          │             10 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 24, 5)          │         23,557 │ layer_normalization_1… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 24, 5)          │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 24, 5)          │              0 │ dropout_4[0][0],       │
│                           │                        │                │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 24, 5)          │             10 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 24, 4)          │             24 │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)  

 Total params: 95,401 (372.66 KB)

 Trainable params: 95,401 (372.66 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
history = model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=100,
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    ]
)

Epoch 1/100
3032/3032 ━━━━━━━━━━━━━━━━━━━━ 85s 18ms/step - loss: 0.0423 - mae: 0.1554 - val_loss: 0.0265 - val_mae: 0.1345 - learning_rate: 1.0000e-04
Epoch 2/100
3032/3032 ━━━━━━━━━━━━━━━━━━━━ 34s 9ms/step - loss: 0.0269 - mae: 0.1318 - val_loss: 0.0269 - val_mae: 0.1361 - learning_rate: 1.0000e-04
Epoch 3/100
3032/3032 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - loss: 0.0262 - mae: 0.1301 - val_loss: 0.0271 - val_mae: 0.1368 - learning_rate: 1.0000e-04
Epoch 4/100
3032/3032 ━━━━━━━━━━━━━━━━━━━━ 26s 8ms/step - loss: 0.0259 - mae: 0.1291 - val_loss: 0.0267 - val_mae: 0.1353 - learning_rate: 1.0000e-04
Epoch 5/100
3032/3032 ━━━━━━━━━━━━━━━━━━━━ 41s 8ms/step - loss: 0.0258 - mae: 0.1287 - val_loss: 0.0272 - val_mae: 0.1371 - learning_rate: 1.0000e-04
Epoch 6/100
3032/3032 ━━━━━━━━━━━━━━━━━━━━ 41s 8ms/step - loss: 0.0258 - mae: 0.1290 - val_loss: 0.0278 - val_mae: 0.1389 - learning_rate: 1.0000e-04
Epoch 7/100
3032/3032 ━━━━━━━━━━━━━━━━━━━━ 42s 9ms/step - loss: 0.0258 - mae: 0.1290 - val_loss: 0.

In [17]:
def predict_next_24_hours(model, input_data):
    predictions = []
    current_input = input_data.copy()

    for _ in range(24):
        # Predict the next step
        prediction = model.predict(current_input)

        # Append the last predicted value to the predictions list
        predictions.append(prediction[0, -1, 0])  # Extract the prediction value

        # Prepare the next input:
        # Shift the time window by one step
        new_input = current_input[:, 1:, :]

        # Create a new row for the next timestep
        new_row = np.zeros((1, 1, current_input.shape[2]))

        # Copy the last known values for all features except AEP_MW
        new_row[0, 0, :-1] = current_input[0, -1, :-1]

        # Set the predicted AEP_MW value
        new_row[0, 0, -1] = prediction[0, -1, 0]

        # Concatenate the new row to the input
        current_input = np.concatenate([new_input, new_row], axis=1)

    # Transform predictions back to original scale
    return scalers['AEP_MW'].inverse_transform(np.array(predictions).reshape(-1, 1))

# Example usage
input_data = X_test[-1].reshape(1, n_timesteps, X_test.shape[2])
predicted_values = predict_next_24_hours(model, input_data)
print(f"Predicted AEP_MW for next 24 hours: {predicted_values.flatten()}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted AEP_MW for next 24 hours: [15509.33  15508.425 15508.381 15508.334 15508.292 1